In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale

In [2]:
path = './UJIIndoorLoc/trainingData2.csv'
train_df = pd.read_csv(path,header=0)
train_ap_features = scale(np.asarray(train_df.iloc[:,:520]).astype(float),axis = 1)
train_df['REFPOINT'] = train_df.apply(lambda row: str(int(row['SPACEID'])) + str(int(row['RELATIVEPOSITION'])),
                                  axis=1)

In [3]:
# blds = np.unique(train_df[['BUILDINGID']]) ##映射Building和Floor到REFPOINT的方法,减少编码量
# flrs = np.unique(train_df[['FLOOR']])
# for bld in blds:
#     for flr in flrs:
#         cond = (train_df['BUILDINGID'] == bld) & (train_df['FLOOR'] == flr)
#         _, idx = np.unique(train_df.loc[cond, 'REFPOINT'], return_inverse=True)  # refer to numpy.unique manual
#         train_df.loc[cond, 'REFPOINT'] = idx
train_labels = np.asarray(pd.get_dummies(train_df['REFPOINT']))
# x_test_utm = (np.asarray(train_df['LONGITUDE'])).reshape(len(train_labels_ref),1)
# y_test_utm = (np.asarray(train_df['LATITUDE'])).reshape(len(train_labels_ref),1)
# train_labels = np.concatenate((train_labels_ref,x_test_utm,y_test_utm), axis=1)
OUTPUT_DIM = train_labels.shape[1]
# train_labels_ref,train_labels_ref.shape,train_labels,train_labels.shape,OUTPUT_DIM
OUTPUT_DIM

213

In [17]:
#划分数据集的方法，  train_test_split  ，不能同时划分标签
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(train_ap_features, test_size=0.2)
# train, val = train_test_split(train, test_size=0.2)
train_test_split = np.random.rand(len(train_ap_features)) < 0.9
x_train_val = train_ap_features[train_test_split]
y_train_val = train_labels[train_test_split]
x_test = train_ap_features[~train_test_split]
y_test = train_labels[~train_test_split]

train_val_split = np.random.rand(len(x_train_val)) < 0.9
x_train = x_train_val[train_val_split]
y_train = y_train_val[train_val_split]
x_val = x_train_val[~train_val_split]
y_val = y_train_val[~train_val_split]

# x_train.shape,y_train.shape
print(len(x_train), '训练数据')   #  训练数据 
print(len(x_val), '验证数据 ')    #  验证数据 
print(len(x_test), '测试数据')    #  测试数据

16194 训练数据
1820 验证数据 
1923 测试数据


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

### building sae

In [5]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
model = Sequential()
sae_hidden_layers= [256,128,64,128,256]
model.add(Dense(sae_hidden_layers[0],input_dim=520,activation='relu',use_bias=False))
for units in sae_hidden_layers[1:]:
    model.add(Dense(units, activation='relu', use_bias=False))
model.add(Dense(520, activation='relu', use_bias=False))
model.compile(optimizer='adam', loss='mse')
model.fit(x_train,x_train,batch_size=64,epochs=20,verbose=1)

Epoch 1/20
254/254 [==============================] - 1s 2ms/step - loss: 0.4721
Epoch 2/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3454
Epoch 3/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3271
Epoch 4/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3186
Epoch 5/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3128
Epoch 6/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3091
Epoch 7/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3064
Epoch 8/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3046
Epoch 9/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3032
Epoch 10/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3019
Epoch 11/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3011
Epoch 12/20
254/254 [==============================] - 1s 2ms/step - loss: 0.3006
Epoch 13/20
254/254 [====

In [6]:
num_to_remove = (len(sae_hidden_layers) + 1) // 2
for i in range(num_to_remove):
    model.pop()

In [7]:
model.summary()#记得加括号

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               133120    
                                                                 
 dense_1 (Dense)             (None, 128)               32768     
                                                                 
 dense_2 (Dense)             (None, 64)                8192      
                                                                 
Total params: 174,080
Trainable params: 174,080
Non-trainable params: 0
_________________________________________________________________


### build and train a complete model with the trained SAE encoder and a new classifier

In [18]:
classifier_hidden_layers = [128,128]
model.add(Dropout(0.1))
for units in classifier_hidden_layers:
    model.add(Dense(units, activation='relu', use_bias=False))
    model.add(Dropout(0.1))
model.add(Dense(OUTPUT_DIM, activation='sigmoid', use_bias=False))  # 'sigmoid' for multi-label classification
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=64, epochs=20, verbose=1)   

Epoch 1/20
254/254 [==============================] - 1s 4ms/step - loss: 0.0772 - accuracy: 0.0184 - val_loss: 0.0262 - val_accuracy: 0.0335
Epoch 2/20
254/254 [==============================] - 1s 4ms/step - loss: 0.0258 - accuracy: 0.0337 - val_loss: 0.0238 - val_accuracy: 0.0451
Epoch 3/20
254/254 [==============================] - 1s 4ms/step - loss: 0.0232 - accuracy: 0.0625 - val_loss: 0.0209 - val_accuracy: 0.1110
Epoch 4/20
254/254 [==============================] - 1s 4ms/step - loss: 0.0202 - accuracy: 0.1279 - val_loss: 0.0181 - val_accuracy: 0.1901
Epoch 5/20
254/254 [==============================] - 1s 4ms/step - loss: 0.0179 - accuracy: 0.1982 - val_loss: 0.0162 - val_accuracy: 0.2610
Epoch 6/20
254/254 [==============================] - 1s 4ms/step - loss: 0.0161 - accuracy: 0.2691 - val_loss: 0.0151 - val_accuracy: 0.3115
Epoch 7/20
254/254 [==============================] - 1s 3ms/step - loss: 0.0145 - accuracy: 0.3409 - val_loss: 0.0136 - val_accuracy: 0.3775
Epoch 

In [19]:
preds = model.predict(x_test, batch_size=32)
results = np.equal(np.argmax(y_test, axis=1), np.argmax(preds, axis=1)).astype(int)
acc = results.mean()
acc

61/61 [==============================] - 0s 1ms/step


0.6510660426417056

In [20]:
e_loss, e_acc = model.evaluate(x_test,y_test)
e_loss,e_acc

61/61 [==============================] - 0s 1ms/step - loss: 0.0096 - accuracy: 0.6511


(0.009556763805449009, 0.6510660648345947)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               133120    
                                                                 
 dense_1 (Dense)             (None, 128)               32768     
                                                                 
 dense_2 (Dense)             (None, 64)                8192      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 128)               8192      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               1

In [11]:
# mask = results.astype(bool)
# mask

array([ True, False, False, ..., False,  True,  True])

In [12]:
# rfps = preds[mask]
# rfps,rfps.shape

(array([[8.92962717e-08, 4.41135606e-04, 1.76552117e-01, ...,
         5.85920601e-09, 4.89989288e-13, 9.34913746e-09],
        [2.76458479e-04, 5.62636118e-11, 3.31597109e-13, ...,
         4.43858231e-12, 4.51149560e-07, 1.03370276e-05],
        [2.42039743e-12, 4.58746484e-07, 2.23110952e-09, ...,
         2.45913776e-04, 3.05888790e-07, 1.63069606e-04],
        ...,
        [5.40047768e-06, 3.99729694e-11, 5.71577857e-07, ...,
         3.87069043e-09, 2.37506369e-12, 1.53328894e-09],
        [1.10493119e-07, 1.20595265e-02, 9.44795012e-01, ...,
         3.24483551e-08, 8.84842866e-09, 5.87115164e-06],
        [9.57750328e-18, 2.87962234e-15, 8.66200244e-17, ...,
         7.01730015e-13, 9.77298646e-13, 1.22221526e-14]], dtype=float32),
 (1282, 213))

In [14]:
# n_success = len(rfps)
# rfps = np.greater_equal(rfps, np.tile(np.amax(rfps, axis=1).reshape(n_success, 1), (1, 213))).astype(
#         int) # set maximum column to 1 and others to 0 (row-wise)
# rfps,rfps.shape,

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 (1282, 213))

In [35]:
# import math
# n_loc_failure = 0
# sum_pos_err = 0.0
# sum_pos_err_weighted = 0.0
# idxs = np.argpartition(rfps, -1)[:, -1:]  # (unsorted) indexes of up to N nearest neighbors
# idxs

array([[58],
       [ 3],
       [71],
       ...,
       [ 1],
       [66],
       [86]], dtype=int64)